In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.metrics import mean_absolute_error
import os
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import ElasticNet

import csv
import os
import json
import numpy as np
from datetime import datetime

In [97]:

folder = '../data/processed'
filename = 'dataset_final.csv'
file_path = os.path.join(folder, filename)

try:
    df = pd.read_csv(
        file_path,
        sep=',',
        decimal='.'
    )
    df = df.sort_values(by='DIA', ignore_index=True)
    print(f"✅ DataFrame cargado exitosamente desde: {file_path}")
    print(f"Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    print(df.head())

except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo en la ruta: {file_path}")
except Exception as e:
    print(f"❌ Error al leer el archivo: {e}")

✅ DataFrame cargado exitosamente desde: ../data/processed\dataset_final.csv
Dimensiones: (1184, 394)

Primeras 5 filas:
          DIA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  \
0  2020-07-01      642.727209            47.145349       69.023256   
1  2020-07-02        7.767254             0.769609        0.798838   
2  2020-07-03        8.801205             0.862593        0.835762   
3  2020-07-04        5.175639             0.439225        0.371077   
4  2020-07-05        7.924665             0.802365        0.717787   

   EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  \
0        0.000000         13.813953        14.578784         0.000000   
1        0.319229          2.358593         4.158962         1.506838   
2        0.260924          1.985462        39.076667         1.448962   
3        0.258048          1.442114         4.348182         1.355238   
4        0.301592          1.664726         5.125920         2.704348   

   EE Linea 4 / Hl  

In [98]:

# 1️⃣ Cargar dataset definitivo
ruta_csv = "../data/processed/X_test_preproc.csv"
X_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/X_train_preproc.csv"
X_train = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_test.csv"
y_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_train.csv"
y_train = pd.read_csv(ruta_csv, sep=',', decimal='.')

y_train = y_train.iloc[:, 0].values.ravel()
y_test  = y_test.iloc[:, 0].values.ravel()

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)




X_train: (828, 40) y_train: (828,)
X_test: (356, 40) y_test: (356,)


In [ ]:
def objective_rf_mae(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=50)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_float("max_features", 0.1, 1.0, log=True)

    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        n_jobs=-1,
        random_state=42
    )

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_mae = []

    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train)):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)

        mae = mean_absolute_error(y_val, preds)
        fold_mae.append(mae)

        # Reportar al pruner
        trial.report(mae, fold_idx)

        # Podar si es necesario
        if trial.should_prune():
            raise optuna.TrialPruned()

    return np.mean(fold_mae)


In [ ]:
def objective_xgb_mae(trial):
    model = xgb.XGBRegressor(
        n_estimators=trial.suggest_int("n_estimators", 100, 2000, step=100),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        max_depth=trial.suggest_int("max_depth", 3, 10),
        subsample=trial.suggest_float("subsample", 0.6, 1.0, step=0.1),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        n_jobs=-1,
        random_state=42,
        verbosity=0
    )

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_mae = []

    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train)):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)

        mae = mean_absolute_error(y_val, preds)
        fold_mae.append(mae)

        trial.report(mae, fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return np.mean(fold_mae)


In [ ]:
def objective_lgbm_mae(trial):
    model = lgb.LGBMRegressor(
        n_estimators=trial.suggest_int("n_estimators", 100, 2000, step=100),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        num_leaves=trial.suggest_int("num_leaves", 20, 150),
        subsample=trial.suggest_float("subsample", 0.6, 1.0, step=0.1),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        n_jobs=-1,
        random_state=42,
        verbose=-1
    )

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_mae = []

    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train)):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)

        mae = mean_absolute_error(y_val, preds)
        fold_mae.append(mae)

        trial.report(mae, fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return np.mean(fold_mae)


In [ ]:
def objective_elasticnet_mae(trial):
    model = ElasticNet(
        alpha=trial.suggest_float("alpha", 1e-3, 10.0, log=True),
        l1_ratio=trial.suggest_float("l1_ratio", 0.0, 1.0),
        random_state=42,
        max_iter=2000
    )

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_mae = []

    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train)):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)

        mae = mean_absolute_error(y_val, preds)
        fold_mae.append(mae)

        trial.report(mae, fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return np.mean(fold_mae)


# 1. Crear los estudios (todos buscan minimizar el MAE)
study_rf_mae = optuna.create_study(direction="minimize")
study_xgb_mae = optuna.create_study(direction="minimize")
study_lgbm_mae = optuna.create_study(direction="minimize")
study_elasticnet_mae = optuna.create_study(direction="minimize")

# 2. Ejecutar las optimizaciones
print("Optimizando Random Forest (MAE)...")
study_rf_mae.optimize(objective_rf_mae, n_trials=50)

print("Optimizando XGBoost (MAE)...")
study_xgb_mae.optimize(objective_xgb_mae, n_trials=75)

print("Optimizando LightGBM (MAE)...")
study_lgbm_mae.optimize(objective_lgbm_mae, n_trials=75)

print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=40)

# 3. Ver los mejores parámetros
print(f"Mejor RF (MAE): {study_rf_mae.best_value} con params {study_rf_mae.best_params}")
print(f"Mejor XGB (MAE): {study_xgb_mae.best_value} con params {study_xgb_mae.best_params}")
print(f"Mejor LGBM (MAE): {study_lgbm_mae.best_value} con params {study_lgbm_mae.best_params}")
print(f"Mejor ElasticNet (MAE): {study_elasticnet_mae.best_value} con params {study_elasticnet_mae.best_params}")

In [103]:

def log_experiment(model_name: str, 
                   model_object: object, 
                   study: optuna.study.Study, 
                   X_train, y_train, X_test, y_test):
    """
    Entrena un modelo con los mejores parámetros de un estudio de Optuna,
    calcula métricas y las guarda en un CSV.
    """
    
    # --- 1. Definir la ruta del log ---
    log_dir = "results"
    log_file = os.path.join(log_dir, "experiment_logs.csv")
    os.makedirs(log_dir, exist_ok=True) # Crea la carpeta si no existe

    # --- 2. Obtener mejores parámetros y valor del estudio ---
    best_params = study.best_params
    best_value = study.best_value # El MAE de CV que encontró Optuna
    
    # --- 3. Instanciar y entrenar el modelo final ---
    # Manejo especial para modelos lineales que necesitan Pipeline
    if "pipeline" in model_name.lower():
        # El pipeline ya está definido en la función objective, 
        # aquí solo re-creamos el mejor.
        if model_name == "Pipeline_ElasticNet":
             # Re-crea el pipeline con los mejores params
             model =ElasticNet(
                    alpha=best_params.get('alpha'), 
                    l1_ratio=best_params.get('l1_ratio'),
                    random_state=42, 
                    max_iter=2000
                )
            
            
    else:
        # Para modelos no-pipeline (RF, XGB, LGBM)
        model = model_object(**best_params, random_state=42, n_jobs=-1)
        # Ajustes para modelos que no les gusta 'n_jobs' o necesitan 'verbose'
        if isinstance(model, (xgb.XGBRegressor, lgb.LGBMRegressor)):
            model.set_params(verbosity=0) if isinstance(model, xgb.XGBRegressor) else None
            model.set_params(verbose=-1) if isinstance(model, lgb.LGBMRegressor) else None

    # Entrenar el modelo final con todos los datos de train
    model.fit(X_train, y_train)

    # --- 4. Calcular Métricas (Train y Test) ---
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    metrics = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_name': model_name,
        'optuna_cv_mae': best_value, # Métrica optimizada
        'mae_train': mean_absolute_error(y_train, y_pred_train),
        'mae_test': mean_absolute_error(y_test, y_pred_test),
        'rmse_train': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'rmse_test': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'r2_train': r2_score(y_train, y_pred_train),
        'r2_test': r2_score(y_test, y_pred_test),
        'hyperparameters': json.dumps(best_params) # Guardar params como string JSON
    }
    
    # --- 5. Escribir en el CSV ---
    file_exists = os.path.isfile(log_file)
    
    with open(log_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=metrics.keys())
        
        if not file_exists:
            writer.writeheader() # Escribir cabecera solo si el archivo es nuevo
            
        writer.writerow(metrics)
        
    print(f"Resultados de '{model_name}' guardados en {log_file}")

In [104]:
# --- 1. Ejecutar el estudio (ejemplo con Random Forest) ---
print("Optimizando Random Forest (MAE)...")
study_rf_mae = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
# (Aquí asumimos que 'objective_rf_mae' está definida)
study_rf_mae.optimize(objective_rf_mae, n_trials=15) 

# --- 2. Registrar el resultado ---
log_experiment(
    model_name="RandomForest",
    model_object=RandomForestRegressor, # Pasa la clase del modelo
    study=study_rf_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 5. Ejecutar el estudio (ejemplo con ElasticNet) ---
print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
# (Aquí asumimos que 'objective_elasticnet_mae' está definida)
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=30)

# --- 6. Registrar el resultado ---
# Nota: 'model_object' es None porque se maneja dentro del 'if'
log_experiment(
    model_name="Pipeline_ElasticNet",
    model_object=None, 
    study=study_elasticnet_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 3. Ejecutar el estudio (ejemplo con XGBoost) ---
print("Optimizando XGBoost (MAE)...")
study_xgb_mae = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
# (Aquí asumimos que 'objective_xgb_mae' está definida)
study_xgb_mae.optimize(objective_xgb_mae, n_trials=5)

# --- 4. Registrar el resultado ---
log_experiment(
    model_name="XGBoost",
    model_object=xgb.XGBRegressor, # Pasa la clase del modelo
    study=study_xgb_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 5. Ejecutar el estudio (ejemplo con LGBM) ---
print("Optimizando LightGBM (MAE)...")
study_lgbm_mae = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study_lgbm_mae.optimize(objective_lgbm_mae, n_trials=10)

# --- 6. Registrar el resultado ---
log_experiment(
    model_name="LightGBM",
    model_object=lgb.LGBMRegressor,  # Pasamos la clase del modelo
    study=study_lgbm_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)


[I 2025-11-15 20:40:15,593] A new study created in memory with name: no-name-878ae403-80c0-41ad-acc1-dface604710d


Optimizando Random Forest (MAE)...


[I 2025-11-15 20:40:21,821] Trial 0 finished with value: -10977.056876404415 and parameters: {'n_estimators': 250, 'max_depth': 42, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.839549239303193}. Best is trial 0 with value: -10977.056876404415.
[I 2025-11-15 20:40:30,633] Trial 1 finished with value: -8970.241214176727 and parameters: {'n_estimators': 750, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 4, 'max_features': 0.7068953429680763}. Best is trial 0 with value: -10977.056876404415.
[I 2025-11-15 20:40:34,512] Trial 2 finished with value: -8946.058383797397 and parameters: {'n_estimators': 400, 'max_depth': 22, 'min_samples_split': 8, 'min_samples_leaf': 8, 'max_features': 0.15143932764268447}. Best is trial 0 with value: -10977.056876404415.
[I 2025-11-15 20:40:44,112] Trial 3 finished with value: -11092.34241235886 and parameters: {'n_estimators': 450, 'max_depth': 20, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': 0.997114044457

Resultados de 'RandomForest' guardados en results\experiment_logs.csv
Optimizando ElasticNet (MAE)...


[I 2025-11-15 20:41:56,253] Trial 2 finished with value: -18043.043782997614 and parameters: {'alpha': 0.06880832755147874, 'l1_ratio': 0.6828877498453998}. Best is trial 2 with value: -18043.043782997614.
c:\Users\franc\Desktop\MostoElMostro\MostoElMostro\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.320e+09, tolerance: 4.230e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\franc\Desktop\MostoElMostro\MostoElMostro\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.398e+12, tolerance: 7.755e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\franc\Desktop\Mos

Resultados de 'Pipeline_ElasticNet' guardados en results\experiment_logs.csv
Optimizando XGBoost (MAE)...


[I 2025-11-15 20:42:20,685] Trial 0 finished with value: -14308.502190085806 and parameters: {'n_estimators': 800, 'learning_rate': 0.23720045769523368, 'max_depth': 7, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 1.8016193468134926e-05, 'reg_lambda': 0.0007089746118842438}. Best is trial 0 with value: -14308.502190085806.
[I 2025-11-15 20:42:23,082] Trial 1 finished with value: -10773.62625600413 and parameters: {'n_estimators': 300, 'learning_rate': 0.04493292675645245, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_alpha': 5.5811534500765627e-05, 'reg_lambda': 0.6075320661292568}. Best is trial 0 with value: -14308.502190085806.
[I 2025-11-15 20:42:59,295] Trial 2 finished with value: -11663.277184253706 and parameters: {'n_estimators': 2000, 'learning_rate': 0.01770500100652277, 'max_depth': 6, 'subsample': 0.6, 'colsample_bytree': 1.0, 'reg_alpha': 2.7802464885467577e-05, 'reg_lambda': 0.02806956384529288}. Best is trial 0 with value: -14308.50219008580

KeyboardInterrupt: 